In [ ]:
# 读取市场状态数据
market_states = {}
for sector in sectors:
    try:
        market_state_file = f"results/sector{sector}/df_market_states.csv"
        if os.path.exists(market_state_file):
            df_market_state = pd.read_csv(market_state_file, index_col=0)
            for idx, row in df_market_state.iterrows():
                date_str = idx if isinstance(idx, str) else idx.strftime('%Y-%m-%d')
                market_states[date_str] = row.to_dict()
    except:
        pass

# 在准备DRL环境时添加市场状态特征
for idx in range(1, len(trade_date)):
    # 现有代码...
    
    # 添加市场状态作为特征
    if trade_date[idx-1] in market_states:
        ms = market_states[trade_date[idx-1]]
        # 创建市场状态特征列
        market_state_features = []
        
        # 将市场状态转换为数值特征
        if ms['state'] == 'Bull':
            bull_indicator = 1
            bear_indicator = 0
            sideways_indicator = 0
        elif ms['state'] == 'Bear':
            bull_indicator = 0
            bear_indicator = 1
            sideways_indicator = 0
        else:  # Sideways
            bull_indicator = 0
            bear_indicator = 0
            sideways_indicator = 1
            
        # 为每一行数据添加市场状态特征
        df_['bull_market'] = bull_indicator
        df_['bear_market'] = bear_indicator
        df_['sideways_market'] = sideways_indicator
        df_['bull_confidence'] = ms.get('bull_confidence', 0)
        df_['bear_confidence'] = ms.get('bear_confidence', 0)
        df_['sideways_confidence'] = ms.get('sideways_confidence', 0)
    
    # 更新环境参数，添加市场状态特征
    if 'bull_market' in df_.columns:
        env_kwargs["tech_indicator_list"] = config.INDICATORS + ['bull_market', 'bear_market', 'sideways_market', 
                                                                'bull_confidence', 'bear_confidence', 'sideways_confidence']
    
    # 训练DRL模型...